<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [55]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [133]:
# TODO - Words, words, mere words, no matter from the heart.
import urllib.request

# I saved the file at the URL locally,then cut out the forematter and endmatter.

fname = "./shakespeare/shakespeare_cleaned.txt"

text = ""

with open(fname) as f:
    lines = f.readlines()

lines = [x.strip() for x in lines]



# I take a subset of lines since the full list is huge
len(lines)

short = int(len(lines) / 50)

few_lines = lines[0:short]

len(few_lines)
text = " ".join(few_lines)


In [134]:
# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

len(chars)

80

In [49]:
# Create lists of sequences and next characters

maxlen = 40
step = 5

# encode each character as an integer
encoded = [char_int[c] for c in text]

# initialize lists to hold data
sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

# We stop at the last sequence
for i in range(0, len(encoded) - maxlen, step):
    
    # append an encoded sequence
    sequences.append(encoded[i : i + maxlen])
    # append a next character
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  11356


In [51]:
# Create X (input matrix) & y (target, which i next char)

X = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

# X  is a 3-space with dims of (number of sequences), length of sequence, count of unique chars
# y is a 2-space with dimensions of (number of sequences), length of sequence
# both are very sparse.

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [52]:
X.shape, y.shape

((11356, 40, 71), (11356, 71))

In [57]:
#Retype for muscle memory

# Instantiate a squential model
model = Sequential()
# Add one LSTM layer
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# Add an Output Layer
model.add(Dense(len(chars), activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [58]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [59]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [131]:
# fit the model

history = model.fit(X, y,
          batch_size=32,
          epochs=5,
          callbacks=[print_callback])

Train on 11356 samples
Epoch 1/5
11328/11356 [============================>.] - ETA: 0s - loss: 1.4097
----- Generating text after Epoch: 0
----- Generating with seed: " shame, Nor thou with public kindness ho"
 shame, Nor thou with public kindness hount ol surwerd sharing. Lrone deterkss with their bre cald it menby serss ofange to bet bugr, The pain me, But berond mime prachict, Oan as the liok) Sin of self wored, Even wren no art  of sull tone, Hine well be of it mabe, A done Asl cerurour on call, my songime. Thou carst the ehious and chirts sill, Thas ce the whery if you remert? I whent fortet my and look loves thos be, or mith ip the wher
11356/11356 [==============================] - 70s 6ms/sample - loss: 1.4104
Epoch 2/5
11328/11356 [============================>.] - ETA: 0s - loss: 1.3856
----- Generating text after Epoch: 1
----- Generating with seed: "ow, For beauty’s pattern to succeeding m"
ow, For beauty’s pattern to succeeding mart’se, of his for, With I te thie do kint,

In [129]:
def shakespeare_chars(how_many):
    """Adds 'how_many' characters to a random string of 'maxlen'
    characters selected from 'text'.  Returns the random string plus
    the generated characters."""
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]
    generated = sentence
    
    for i in range(how_many):
        
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    return generated

In [130]:
shakespeare_chars(100)

'In other accents do this praise confound: Afrming dosuld dath thas braving worst will, A dese thought muth swill, in hee you arkever, thy wr'

In [132]:
old_model = model

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN